<a href="https://colab.research.google.com/github/Akhorramrouz/My-body-Your-choice/blob/main/Fine_Tune_Persian_bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install required packages
!pip install -q transformers
!pip install -q hazm
!pip install -q clean-text[gpl]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 371.7/371.7 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 14.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 57.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 993.5/993.5 kB 52.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.7/227.7 kB 21.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.4/175.4 kB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# Import required packages

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from transformers import glue_convert_examples_to_features

from sklearn.metrics import f1_score
from sklearn.utils import shuffle

import hazm
from cleantext import clean

import plotly.express as px
import plotly.graph_objects as go

from tqdm.notebook import tqdm

import os
import re
import json
import copy
import collections

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd "/content/drive/MyDrive/My-body-Your-choice/"

/content/drive/MyDrive/My-body-Your-choice


### Load the data using Pandas

In [ ]:
data = pd.read_csv('data/entailment_labled_GPT/unique_all_topics_enatilment_labled_by_GPT4_with_keyword_abortion_with_forced_label.csv', encoding='utf-8',index_col=0)
data.rename(columns={"text":"comment",'Forced_by_anyone':'rate'}, inplace=True)
# data = data.loc[data.label.str.contains('entailment')]
data = data[['comment', 'rate']]
print(data.shape)
data.head()

(1482, 2)


,comment,rate
0,\nکسی بخواد از پسری که قول ازدواج داده اومده ج...,1
1,\nسلام دیشب داشتم با شوهرم بحث میکردیم درمورد ...,1
2,\nسلام.من بعد ۱۳ سال یکماهی هست تصمیمم جدیه بر...,1
3,\nکسی رو دیدید که بچه شو سقط کنه دختر دایی من ...,1
4,\nدوستان من با نامزدم رابطه داشتم امروز فهمیدم...,1


In [ ]:
print(data.rate.value_counts())

0    1360
1     122
Name: rate, dtype: int64


In [ ]:
# print data information
print('data information')
print(data.info(), '\n')

# print missing values information
print('missing values stats')
print(data.isnull().sum(), '\n')

# print some missing values
print('some missing values')
print(data[data['rate'].isnull()].iloc[:5], '\n')

data information
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1482 entries, 0 to 1481
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   comment  1482 non-null   object
 1   rate     1482 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 34.7+ KB
None 

missing values stats
comment    0
rate       0
dtype: int64 

some missing values
Empty DataFrame
Columns: [comment, rate]
Index: [] 



In [ ]:
# calculate the length of comments based on their words
data['comment_len_by_words'] = data['comment'].apply(lambda t: len(hazm.word_tokenize(t)))

In [ ]:
min_max_len = data["comment_len_by_words"].min(), data["comment_len_by_words"].max()
print(f'Min: {min_max_len[0]} \tMax: {min_max_len[1]}')

Min: 5 	Max: 972


In [ ]:
def data_gl_than(data, less_than=100.0, greater_than=0.0, col='comment_len_by_words'):
    data_length = data[col].values

    data_glt = sum([1 for length in data_length if greater_than < length <= less_than])

    data_glt_rate = (data_glt / len(data_length)) * 100

    print(f'Texts with word length of greater than {greater_than} and less than {less_than} includes {data_glt_rate:.2f}% of the whole!')

In [ ]:
data_gl_than(data, 256, 5)

Texts with word length of greater than 5 and less than 256 includes 92.24% of the whole!


In [ ]:
minlim, maxlim = 3, 256

In [ ]:
# remove comments with the length of fewer than three words
data['comment_len_by_words'] = data['comment_len_by_words'].apply(lambda len_t: len_t if minlim < len_t <= maxlim else None)
data = data.dropna(subset=['comment_len_by_words'])
data = data.reset_index(drop=True)

In [ ]:
fig = go.Figure()

fig.add_trace(go.Histogram(
    x=data['comment_len_by_words']
))

fig.update_layout(
    title_text='Distribution of word counts within comments',
    xaxis_title_text='Word Count',
    yaxis_title_text='Frequency',
    bargap=0.2,
    bargroupgap=0.2)

fig.show()

In [ ]:
unique_rates = list(sorted(data['rate'].unique()))
print(f'We have #{len(unique_rates)}: {unique_rates}')

We have #2: [0, 1]


In [ ]:
fig = go.Figure()

groupby_rate = data.groupby('rate')['rate'].count()

fig.add_trace(go.Bar(
    x=list(sorted(groupby_rate.index)),
    y=groupby_rate.tolist(),
    text=groupby_rate.tolist(),
    textposition='auto'
))

fig.update_layout(
    title_text='Distribution of rate within comments',
    xaxis_title_text='Rate',
    yaxis_title_text='Frequency',
    bargap=0.2,
    bargroupgap=0.2)

fig.show()

For simplicity, I transformed the rate in a range of 0.0 to 5.0 to a binary form of negative (0) or positive (1) with a threshold. If the rate is less than 3.0, it labeled as negative otherwise specified as positive.

In [ ]:
def rate_to_label(rate, threshold=0.5):
    if rate <= threshold:
        return 'negative'
    else:
        return 'positive'


data['label'] = data['rate'].apply(lambda t: rate_to_label(t, 0.5))
labels = list(sorted(data['label'].unique()))
data.head()

,comment,rate,comment_len_by_words,label
0,\nکسی بخواد از پسری که قول ازدواج داده اومده ج...,1,66.0,positive
1,\nسلام دیشب داشتم با شوهرم بحث میکردیم درمورد ...,1,255.0,positive
2,\nکسی رو دیدید که بچه شو سقط کنه دختر دایی من ...,1,65.0,positive
3,\nدوستان من با نامزدم رابطه داشتم امروز فهمیدم...,1,32.0,positive
4,\nحس میکنم شوهرم با راهنمایی و توصیه یه دختر ...,1,29.0,positive


Cleaning is the final step in this section. Our cleaned method includes these steps:

- fixing unicodes
- removing specials like a phone number, email, url, new lines, ...
- cleaning HTMLs
- normalizing
- removing emojis

In [ ]:
def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', raw_html)
    return cleantext


def remove_emojis(text):
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642"
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)
    return emoji_pattern.sub(r'', text)

def remove_punctuations(text):
    punctuation_pattern = re.compile(r'[^\w\s]')
    return punctuation_pattern.sub(r'', text)


def clean(text):
  text = remove_emojis(text)
  text = remove_punctuations(text)
  text = cleanhtml(text)
  text = text.replace("\n"," ").strip()
  return text

In [ ]:
# Vectorized cleaning operation
data['cleaned_comment'] = data['comment'].apply(lambda x: clean(x))

# Tokenize and count words in comments
data['cleaned_comment_len_by_words'] = data['cleaned_comment'].apply(lambda t: len(hazm.word_tokenize(t)))

# Filter based on word count limits (assuming minlim and maxlim are defined)
data = data[(data['cleaned_comment_len_by_words'] > minlim) & (data['cleaned_comment_len_by_words'] <= maxlim)]

# Reset index after dropping rows
data = data.reset_index(drop=True)

# Display the first few rows
print(data.head())


                                             comment  rate  \
0  \nکسی بخواد از پسری که قول ازدواج داده اومده ج...     1   
1  \nسلام دیشب داشتم با شوهرم بحث میکردیم درمورد ...     1   
2  \nکسی رو دیدید که بچه شو سقط کنه دختر دایی من ...     1   
3  \nدوستان من با نامزدم رابطه داشتم امروز فهمیدم...     1   
4  \nحس میکنم شوهرم با راهنمایی و توصیه یه دختر  ...     1   

   comment_len_by_words     label  \
0                  66.0  positive   
1                 255.0  positive   
2                  65.0  positive   
3                  32.0  positive   
4                  29.0  positive   

                                     cleaned_comment  \
0  کسی بخواد از پسری که قول ازدواج داده اومده جلو...   
1  سلام دیشب داشتم با شوهرم بحث میکردیم درمورد خی...   
2  کسی رو دیدید که بچه شو سقط کنه دختر دایی من ۳ ...   
3  دوستان من با نامزدم رابطه داشتم امروز فهمیدم ب...   
4  حس میکنم شوهرم با راهنمایی و توصیه یه دختر  جن...   

   cleaned_comment_len_by_words  
0                            64  

In [ ]:
data.label.value_counts()

negative    1258
positive     108
Name: label, dtype: int64

In [ ]:
data = data[['cleaned_comment', 'label']]
data.columns = ['comment', 'label']
data.head()

,comment,label
0,کسی بخواد از پسری که قول ازدواج داده اومده جلو...,positive
1,سلام دیشب داشتم با شوهرم بحث میکردیم درمورد خی...,positive
2,کسی رو دیدید که بچه شو سقط کنه دختر دایی من ۳ ...,positive
3,دوستان من با نامزدم رابطه داشتم امروز فهمیدم ب...,positive
4,حس میکنم شوهرم با راهنمایی و توصیه یه دختر جن...,positive


In [ ]:
print(f'We have #{len(labels)} labels: {labels}')

We have #2 labels: ['negative', 'positive']


### Handling Unbalanced Data

In [ ]:
fig = go.Figure()

groupby_label = data.groupby('label')['label'].count()

fig.add_trace(go.Bar(
    x=list(sorted(groupby_label.index)),
    y=groupby_label.tolist(),
    text=groupby_label.tolist(),
    textposition='auto'
))

fig.update_layout(
    title_text='Distribution of label within comments [DATA]',
    xaxis_title_text='Label',
    yaxis_title_text='Frequency',
    bargap=0.2,
    bargroupgap=0.2)

fig.show()

Again, for making things simple. We cut the dataset randomly based on the fewer label, the negative class.

In [ ]:
negative_data = data[data['label'] == 'negative']
positive_data = data[data['label'] == 'positive']

cutting_point =int( min(len(negative_data), len(positive_data)) * 2.5)

if cutting_point <= len(negative_data):
    negative_data = negative_data.sample(n=cutting_point).reset_index(drop=True)

if cutting_point <= len(positive_data):
    positive_data = positive_data.sample(n=cutting_point).reset_index(drop=True)

new_data = pd.concat([negative_data, positive_data])
new_data = new_data.sample(frac=1).reset_index(drop=True)
new_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 378 entries, 0 to 377
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   comment  378 non-null    object
 1   label    378 non-null    object
dtypes: object(2)
memory usage: 6.0+ KB


In [ ]:
fig = go.Figure()

groupby_label = new_data.groupby('label')['label'].count()

fig.add_trace(go.Bar(
    x=list(sorted(groupby_label.index)),
    y=groupby_label.tolist(),
    text=groupby_label.tolist(),
    textposition='auto'
))

fig.update_layout(
    title_text='Distribution of label within comments [NEW DATA]',
    xaxis_title_text='Label',
    yaxis_title_text='Frequency',
    bargap=0.2,
    bargroupgap=0.2)

fig.show()

In [ ]:
new_data.head()

,comment,label
0,من بخاطر مریضیم باهاش بهم زدم سرطان معده دارما...,negative
1,اگه در جریان تاپبکام نیستین دوس داشتین تاپیکام...,positive
2,بچم ۶ ماهشه فوق العاده بدلج و بدخوابه مادرمو...,negative
3,سلام من بارداری سومم هست یه پسر و دختر دارم ا...,negative
4,بچه دوم رو ناخواسته باردارم شوهرم جنگ و دعوا ر...,positive


## Train,Validation,Test split

To achieve a globalized model, we need to split the cleaned dataset into train, valid, test sets due to size of the data. In this tutorial, I have considered a rate of **0.1** for both *valid*, *test* sets. For splitting, I use `train_test_split` provided by Sklearn package with stratifying on the label for preserving the distribution balance.

In [ ]:
new_data['label_id'] = new_data['label'].apply(lambda t: labels.index(t))

train, test = train_test_split(new_data, test_size=0.1, random_state=1, stratify=new_data['label'])
train, valid = train_test_split(train, test_size=0.1, random_state=1, stratify=train['label'])

train = train.reset_index(drop=True)
valid = valid.reset_index(drop=True)
test = test.reset_index(drop=True)

x_train, y_train = train['comment'].values.tolist(), train['label_id'].values.tolist()
x_valid, y_valid = valid['comment'].values.tolist(), valid['label_id'].values.tolist()
x_test, y_test = test['comment'].values.tolist(), test['label_id'].values.tolist()

print(train.shape)
print(valid.shape)
print(test.shape)

(306, 3)
(34, 3)
(38, 3)


## TensorFlow

In [ ]:
from transformers import BertConfig, BertTokenizer
from transformers import TFBertModel, TFBertForSequenceClassification
from transformers import glue_convert_examples_to_features

import tensorflow as tf

### Configuration

In [ ]:
# general config
MAX_LEN = 128
TRAIN_BATCH_SIZE = 16
VALID_BATCH_SIZE = 16
TEST_BATCH_SIZE = 16

EPOCHS = 5
EEVERY_EPOCH = 1000
LEARNING_RATE = 2e-5
CLIP = 0.0

MODEL_NAME_OR_PATH = 'HooshvareLab/bert-fa-base-uncased'
OUTPUT_PATH = '/content/bert-fa-base-uncased-sentiment-taaghceh/pytorch_model.bin'

os.makedirs(os.path.dirname(OUTPUT_PATH), exist_ok=True)

In [ ]:
label2id = {label: i for i, label in enumerate(labels)}
id2label = {v: k for k, v in label2id.items()}

print(f'label2id: {label2id}')
print(f'id2label: {id2label}')

label2id: {'negative': 0, 'positive': 1}
id2label: {0: 'negative', 1: 'positive'}


In [ ]:
tokenizer = BertTokenizer.from_pretrained(MODEL_NAME_OR_PATH)
config = BertConfig.from_pretrained(
    MODEL_NAME_OR_PATH, **{
        'label2id': label2id,
        'id2label': id2label,
    })

print(config.to_json_string())

{
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "negative",
    "1": "positive"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "negative": 0,
    "positive": 1
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.35.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 100000
}



### Input Embeddings / Dataset

In [ ]:
class InputExample:
    """ A single example for simple sequence classification. """

    def __init__(self, guid, text_a, text_b=None, label=None):
        """ Constructs a InputExample. """
        self.guid = guid
        self.text_a = text_a
        self.text_b = text_b
        self.label = label


def make_examples(tokenizer, x, y=None, maxlen=128, output_mode="classification", is_tf_dataset=True):
    examples = []
    y = y if isinstance(y, list) or isinstance(y, np.ndarray) else [None] * len(x)

    for i, (_x, _y) in tqdm(enumerate(zip(x, y)), position=0, total=len(x)):
        guid = "%s" % i
        label = int(_y)

        if isinstance(_x, str):
            text_a = _x
            text_b = None
        else:
            assert len(_x) == 2
            text_a = _x[0]
            text_b = _x[1]

        examples.append(InputExample(guid=guid, text_a=text_a, text_b=text_b, label=label))

    features = glue_convert_examples_to_features(
        examples,
        tokenizer,
        maxlen,
        output_mode=output_mode,
        label_list=list(np.unique(y)))

    all_input_ids = []
    all_attention_masks = []
    all_token_type_ids = []
    all_labels = []

    for f in tqdm(features, position=0, total=len(examples)):
        if is_tf_dataset:
            all_input_ids.append(tf.constant(f.input_ids))
            all_attention_masks.append(tf.constant(f.attention_mask))
            all_token_type_ids.append(tf.constant(f.token_type_ids))
            all_labels.append(tf.constant(f.label))
        else:
            all_input_ids.append(f.input_ids)
            all_attention_masks.append(f.attention_mask)
            all_token_type_ids.append(f.token_type_ids)
            all_labels.append(f.label)

    if is_tf_dataset:
        dataset = tf.data.Dataset.from_tensor_slices(({
            'input_ids': all_input_ids,
            'attention_mask': all_attention_masks,
            'token_type_ids': all_token_type_ids
        }, all_labels))

        return dataset, features

    xdata = [np.array(all_input_ids), np.array(all_attention_masks), np.array(all_token_type_ids)]
    ydata = all_labels

    return [xdata, ydata], features

In [ ]:
train_dataset_base, train_examples = make_examples(tokenizer, x_train, y_train, maxlen=128)
valid_dataset_base, valid_examples = make_examples(tokenizer, x_valid, y_valid, maxlen=128)

test_dataset_base, test_examples = make_examples(tokenizer, x_test, y_test, maxlen=128)
[xtest, ytest], test_examples = make_examples(tokenizer, x_test, y_test, maxlen=128, is_tf_dataset=False)

  0%|          | 0/306 [00:00<?, ?it/s]

  0%|          | 0/306 [00:00<?, ?it/s]

  0%|          | 0/34 [00:00<?, ?it/s]

  0%|          | 0/34 [00:00<?, ?it/s]

  0%|          | 0/38 [00:00<?, ?it/s]

  0%|          | 0/38 [00:00<?, ?it/s]

  0%|          | 0/38 [00:00<?, ?it/s]

  0%|          | 0/38 [00:00<?, ?it/s]

In [ ]:
for value in train_dataset_base.take(1):
    print(f'     input_ids: {value[0]["input_ids"]}')
    print(f'attention_mask: {value[0]["attention_mask"]}')
    print(f'token_type_ids: {value[0]["token_type_ids"]}')
    print(f'        target: {value[1]}')

     input_ids: [    2 78508  2008 11337  5334  2054  3199  5991 14067  5501  1379  5971
  2015  2861  3595 34860 40233  2015  5983 19436  1379  2799 15868  9094
  3286  2842 77988  2003  2822  1379  3355 27633  9795  5505 44753  5165
  1379  3711  9186  2822 95812  3805  7085 20128  1353 31751  1426  3130
  8417  8725 12702  1379  2842  2791 85072  3130  9186 22673 79933  2015
  3517 62106  5589  2008 12593  5441 19994 19328  5682 20516  8454 20516
  2003  3679  2008  9090 12987  1379  3235  2871  2848  9313 25200  2008
  9313 22067  3777 30838 12483 12593  5441 19994 54339 16604  7626  2791
  4826 58797  5803 12593  5505  5268 24961  3206 10631  2848  5441  4244
  3327  6041  2015  4467 19451  3789  3442 51015  2935  3362 38979 17802
  3046  1379 17802 15553  2054 42818  2958     4]
attention_mask: [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1

In [ ]:
def get_training_dataset(dataset, batch_size):
    dataset = dataset.repeat()
    dataset = dataset.shuffle(2048)
    dataset = dataset.batch(batch_size)

    return dataset

def get_validation_dataset(dataset, batch_size):
    dataset = dataset.batch(batch_size)

    return dataset

In [ ]:
train_dataset = get_training_dataset(train_dataset_base, TRAIN_BATCH_SIZE)
valid_dataset = get_training_dataset(valid_dataset_base, VALID_BATCH_SIZE)

train_steps = len(train_examples) // TRAIN_BATCH_SIZE
valid_steps = len(valid_examples) // VALID_BATCH_SIZE

train_steps, valid_steps

(19, 2)

### Model

In [ ]:
def build_model(model_name, config, learning_rate=3e-5):
    model = TFBertForSequenceClassification.from_pretrained(model_name, config=config)

    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
    model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

    return model

In [ ]:
model = build_model(MODEL_NAME_OR_PATH, config, learning_rate=LEARNING_RATE)

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at HooshvareLab/bert-fa-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Training

In [ ]:
%%time

r = model.fit(
    train_dataset,
    validation_data=valid_dataset,
    steps_per_epoch=train_steps,
    validation_steps=valid_steps,
    epochs=EPOCHS,
    verbose=1)

final_accuracy = r.history['val_accuracy']
print('FINAL ACCURACY MEAN: ', np.mean(final_accuracy))

Epoch 1/5
19/19 [==============================] - 60s 467ms/step - loss: 0.5404 - accuracy: 0.7204 - val_loss: 0.6546 - val_accuracy: 0.6250
Epoch 2/5
19/19 [==============================] - 5s 280ms/step - loss: 0.4499 - accuracy: 0.8026 - val_loss: 0.6063 - val_accuracy: 0.6875
Epoch 3/5
19/19 [==============================] - 4s 225ms/step - loss: 0.2893 - accuracy: 0.8783 - val_loss: 0.3926 - val_accuracy: 0.8750
Epoch 4/5
19/19 [==============================] - 4s 225ms/step - loss: 0.1827 - accuracy: 0.9145 - val_loss: 0.9041 - val_accuracy: 0.7188
Epoch 5/5
19/19 [==============================] - 6s 292ms/step - loss: 0.1164 - accuracy: 0.9704 - val_loss: 0.4645 - val_accuracy: 0.8125
FINAL ACCURACY MEAN:  0.74375
CPU times: user 1min 14s, sys: 1.58 s, total: 1min 15s
Wall time: 1min 19s


In [ ]:
import tensorflow as tf
from transformers import TFBertForSequenceClassification

def build_model(model_name, config, learning_rate=3e-5):
    model = TFBertForSequenceClassification.from_pretrained(model_name, config=config)

    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
    model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

    return model

# Assuming MODEL_NAME_OR_PATH, config, LEARNING_RATE, train_dataset, valid_dataset, train_steps, valid_steps, and EPOCHS are defined
model = build_model(MODEL_NAME_OR_PATH, config, learning_rate=LEARNING_RATE)

# Callback to save the best model
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    'best_model.h5', monitor='val_loss', save_best_only=True, save_weights_only=True)


r = model.fit(
    train_dataset,
    validation_data=valid_dataset,
    steps_per_epoch=train_steps,
    validation_steps=valid_steps,
    epochs=EPOCHS,
    verbose=1,
    callbacks=[checkpoint_callback])

# Load the best model
model.load_weights('best_model.h5')

# Evaluate the model
final_accuracy = model.evaluate(valid_dataset, steps=valid_steps)[1]
print('FINAL ACCURACY: ', final_accuracy)


All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at HooshvareLab/bert-fa-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/5
19/19 [==============================] - 76s 1s/step - loss: 0.6426 - accuracy: 0.6645 - val_loss: 0.6248 - val_accuracy: 0.6875
Epoch 2/5
19/19 [==============================] - 8s 425ms/step - loss: 0.5286 - accuracy: 0.7368 - val_loss: 0.3505 - val_accuracy: 0.8125
Epoch 3/5
19/19 [==============================] - 5s 268ms/step - loss: 0.3131 - accuracy: 0.8750 - val_loss: 0.8791 - val_accuracy: 0.6875
Epoch 4/5
19/19 [==============================] - 4s 217ms/step - loss: 0.2286 - accuracy: 0.9309 - val_loss: 0.6304 - val_accuracy: 0.7500
Epoch 5/5
2/2 [==============================] - 0s 108ms/step - loss: 0.5031 - accuracy: 0.8125
FINAL ACCURACY:  0.8125


### Evaluation / Prediction

In [ ]:
ev = model.evaluate(test_dataset_base.batch(TEST_BATCH_SIZE))
print()
print(f'Evaluation: {ev}')
print()

predictions = model.predict(xtest)
ypred = predictions[0].argmax(axis=-1).tolist()

print()
print(classification_report(ytest, ypred, target_names=labels))
print()

print(f'F1: {f1_score(ytest, ypred, average="weighted")}')

3/3 [==============================] - 0s 74ms/step - loss: 0.4760 - accuracy: 0.8158

Evaluation: [0.47603029012680054, 0.8157894611358643]

2/2 [==============================] - 4s 47ms/step

              precision    recall  f1-score   support

    negative       0.92      0.81      0.86        27
    positive       0.64      0.82      0.72        11

    accuracy                           0.82        38
   macro avg       0.78      0.82      0.79        38
weighted avg       0.84      0.82      0.82        38


F1: 0.8214241486068111


In [ ]:
# save the model

model.save_pretrained(os.path.dirname("/content/drive/MyDrive/My-body-Your-choice/model_forced_abortion"))

In [ ]:
loaded_model = TFBertForSequenceClassification.from_pretrained(os.path.dirname("/content/drive/MyDrive/My-body-Your-choice/model_forced_abortion"))

Some layers from the model checkpoint at /content/drive/MyDrive/My-body-Your-choice were not used when initializing TFBertForSequenceClassification: ['dropout_113']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at /content/drive/MyDrive/My-body-Your-choice.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


In [ ]:
loaded_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE), loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))
ev = loaded_model.evaluate(test_dataset_base.batch(TEST_BATCH_SIZE))


predictions = loaded_model.predict(xtest)
ypred = predictions[0].argmax(axis=-1).tolist()

print()
print(classification_report(ytest, ypred, target_names=labels))
print()

print(f'F1: {f1_score(ytest, ypred, average="weighted")}')

3/3 [==============================] - 9s 80ms/step - loss: 0.4760

Evaluation: 0.47603029012680054

2/2 [==============================] - 5s 63ms/step

              precision    recall  f1-score   support

    negative       0.92      0.81      0.86        27
    positive       0.64      0.82      0.72        11

    accuracy                           0.82        38
   macro avg       0.78      0.82      0.79        38
weighted avg       0.84      0.82      0.82        38


F1: 0.8214241486068111
